In [30]:
import numpy as np
import cv2

In [31]:
ball_color = "red"

color_dist = {
    #'red':{'Lower':np.array([150,43,46]),'Upper':np.array([180,255,255])},
    'red':{'Lower':np.array([140,43,46]),'Upper':np.array([180,255,255])},
    'blue':{'Lower':np.array([70,70,70]),'Upper':np.array([124,255,255])},
    'green':{'Lower':np.array([35,43,35]),'Upper':np.array([90,255,255])},
}

In [32]:
cap = cv2.VideoCapture(0)
cv2.namedWindow("camera",cv2.WINDOW_AUTOSIZE)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        if frame is not None:
            gs_frame = cv2.GaussianBlur(frame,(5,5),0) #高斯模糊
            hsv = cv2.cvtColor(gs_frame,cv2.COLOR_BGR2HSV) #转化为hsv图像
            erode_hsv = cv2.erode(hsv,None,iterations=2) #腐蚀
            inRange_hsv = cv2.inRange(erode_hsv,color_dist[ball_color]['Lower'],color_dist[ball_color]['Upper'])
            cnts = cv2.findContours(inRange_hsv.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]

            c = max(cnts,key=cv2.contourArea)
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            cv2.drawContours(frame,[np.int0(box)],-1,(0,255,255),2)

            cv2.putText(frame,str(np.int0(box[0][0]))+','+str(np.int0(box[0][1])),(np.int0(box[0][0]),np.int0(box[0][1])),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(0,255,255),1)
            cv2.putText(frame,str(np.int0(box[1][0]))+','+str(np.int0(box[1][1])),(np.int0(box[1][0]),np.int0(box[1][1])),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(0,255,255),1)
            cv2.putText(frame,str(np.int0(box[2][0]))+','+str(np.int0(box[2][1])),(np.int0(box[2][0]),np.int0(box[2][1])),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(0,255,255),1)
            cv2.putText(frame,str(np.int0(box[3][0]))+','+str(np.int0(box[3][1])),(np.int0(box[3][0]),np.int0(box[3][1])),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(0,255,255),1)
            cv2.putText(frame,str(np.int0(box[3][0]/2+box[1][0]/2))+','+str(np.int0(box[3][1]/2+box[1][1]/2)),(np.int0(box[3][0]/2+box[1][0]/2),np.int0(box[3][1]/2+box[1][1]/2)),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(0,255,255),1)
            
            cv2.imshow('camera',frame)
            cv2.waitKey(1)
        else:
            print("No image")
    else:
        print("Cannot read camera!")

KeyboardInterrupt: 

In [38]:
cv2.destroyAllWindows()
cv2.waitKey(0)
cap.release()
print("Release camera")

Release camera


In [37]:
# 设置要识别的颜色
ball_color = "red"  # 可更改为 "red", "blue" 或 "green"

# 颜色阈值定义
color_dist = {
    'red': [
        {'Lower': np.array([0, 100, 100]), 'Upper': np.array([10, 255, 255])},
        {'Lower': np.array([160, 100, 100]), 'Upper': np.array([180, 255, 255])}
    ],
    'blue': {'Lower': np.array([70, 70, 70]), 'Upper': np.array([124, 255, 255])},
    'green': {'Lower': np.array([35, 43, 35]), 'Upper': np.array([90, 255, 255])},
}

cap = cv2.VideoCapture(0)
cv2.namedWindow("camera", cv2.WINDOW_AUTOSIZE)
if ball_color=="red": cv2.namedWindow("mask", cv2.WINDOW_AUTOSIZE)  # 新增mask窗口用于调试

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        if frame is not None:
            # 预处理
            gs_frame = cv2.GaussianBlur(frame, (5, 5), 0)  # 高斯模糊
            hsv = cv2.cvtColor(gs_frame, cv2.COLOR_BGR2HSV)  # 转化为HSV图像
            erode_hsv = cv2.erode(hsv, None, iterations=2)  # 腐蚀
            
            # 颜色阈值处理
            if ball_color == 'red':
                # 红色需要处理两个区间
                lower1 = color_dist[ball_color][0]['Lower']
                upper1 = color_dist[ball_color][0]['Upper']
                lower2 = color_dist[ball_color][1]['Lower']
                upper2 = color_dist[ball_color][1]['Upper']
                
                mask1 = cv2.inRange(erode_hsv, lower1, upper1)
                mask2 = cv2.inRange(erode_hsv, lower2, upper2)
                inRange_hsv = cv2.bitwise_or(mask1, mask2)
                inRange_hsv = cv2.dilate(inRange_hsv, None, iterations=1)  # 膨胀操作
            else:
                # 其他颜色正常处理
                inRange_hsv = cv2.inRange(erode_hsv, 
                                         color_dist[ball_color]['Lower'], 
                                         color_dist[ball_color]['Upper'])
            
            # 显示mask用于调试
            if ball_color=="red": cv2.imshow('mask', inRange_hsv)
            
            # 查找轮廓
            cnts = cv2.findContours(inRange_hsv.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
            
            if cnts:
                # 面积过滤，去除小噪点
                cnts = [c for c in cnts if cv2.contourArea(c) > 500]
                
                if cnts:
                    c = max(cnts, key=cv2.contourArea)
                    rect = cv2.minAreaRect(c)
                    box = cv2.boxPoints(rect)
                    
                    # 绘制边界框
                    cv2.drawContours(frame, [np.int0(box)], -1, (0, 255, 255), 2)
                    
                    # 绘制四个角点坐标
                    for i, point in enumerate(box):
                        x, y = np.int0(point)
                        cv2.putText(frame, f"{x},{y}", (x, y), 
                                   cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.75, (0, 255, 255), 1)
                    
                    # 计算并绘制中心点
                    center_x = int((box[3][0]/2 + box[1][0]/2))
                    center_y = int((box[3][1]/2 + box[1][1]/2))
                    cv2.putText(frame, f"{center_x},{center_y}", (center_x, center_y),
                               cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.75, (0, 255, 255), 1)
            
            cv2.imshow('camera', frame)
            key = cv2.waitKey(1)
            if key == 27:  # ESC键退出
                break
        else:
            print("No image")
    else:
        print("Cannot read camera!")

cap.release()
cv2.destroyAllWindows()